In [1]:
!git clone https://github.com/openai/CLIP.git
%cd CLIP
!pip install -e .

Cloning into 'CLIP'...
remote: Enumerating objects: 256, done.
remote: Total 256 (delta 0), reused 0 (delta 0), pack-reused 256 (from 1)
Receiving objects: 100% (256/256), 8.93 MiB | 14.72 MiB/s, done.
Resolving deltas: 100% (133/133), done.
/content/CLIP
Obtaining file:///content/CLIP
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.8 MB/s eta 0:00:00
  Running setup.py develop for clip


In [2]:
!pip install laion-clap
!pip install clip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.6 MB/s eta 0:00:00
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12065 sha256=b6200bad4c4160460885855ac715e620517d36eddd6437248740140e49fa61f1
  Stored in directory: /root/.cache/pip/wheels/a5/4d/c7/f3cf0f75c746c219090060131fe00f1523cc2c5484991f4030
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=73d8f2430439cb6e4882dd93e0f8de5523bd57ba40eead3c1b72e6acb09cecaf
  Stored in directory: /root/.cache/pip/wheels/01/46/3b/e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built progressbar wget
  Attempting uninstall: numpy
    Found existing ins

In [2]:
!pip install torch

In [3]:
import clip
import laion_clap
import torch

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
clip_model, preprocess = clip.load("ViT-B/16", device=device)

In [65]:
image_model = clip_model.visual
img_dim = image_model.output_dim

In [66]:
for p in clip_model.parameters():
  p.requires_grad=False

In [67]:
clap_model = laion_clap.CLAP_Module(enable_fusion=False)
clap_model.load_ckpt()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm1.weight 	 Loaded
audio_branch.layers.0.blocks.0.norm1.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.relative_position_bias_table 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm2.we

In [68]:
clap_model.load_ckpt()

Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm1.weight 	 Loaded
audio_branch.layers.0.blocks.0.norm1.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.relative_position_bias_table 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm2.we

In [70]:
for p in clap_model.parameters():
  p.requires_grad=False

In [71]:
import torch.nn as nn

shared_dim = 256
num_epochs = 2

image_proj = nn.Linear(image_dim, shared_dim).to(device)
audio_proj = nn.Linear(512, shared_dim).to(device)


In [83]:
def encod_img(image):
  with torch.no_grad():
    img_feat = clip_model.encode_image(image=image)

  img_emb = image_proj(img_feat)
  return img_emb/img_emb.norm(dim=-1, keepdim=True)

In [86]:
def encod_audio(audio_batch):
    """
    audio_batch: Tensor of shape [B, 1, T]
    returns: Tensor of shape [B, D]
    """
    embs = []

    for i in range(audio_batch.size(0)):
        audio_wave = audio_batch[i]   # shape: [1, T]

        with torch.no_grad():
            aud_feat = clap_model.get_audio_embedding_from_data(
                audio_wave,
                use_tensor=True
            )  # shape: [1, audio_dim]

        aud_emb = audio_proj(aud_feat)  # [1, proj_dim]
        aud_emb = aud_emb / aud_emb.norm(dim=-1, keepdim=True)

        embs.append(aud_emb)

    return torch.cat(embs, dim=0)  # [B, proj_dim]


In [87]:
optimizer = torch.optim.Adam(
    list(image_proj.parameters()) +
    list(audio_proj.parameters()),
    lr=1e-3
)


In [88]:
pairs = [
    ("/content/Dataset/Images/1.jpg", "/content/data/Audios/1.wav"),
    ("/content/Dataset/Images/2.jpg", "/content/data/Audios/2.wav"),
    ("/content/Dataset/Images/3.jpg", "/content/data/Audios/3.wav"),
    ("/content/Dataset/Images/4.jpg", "/content/data/Audios/4.wav"),
    ("/content/Dataset/Images/5.jpg", "/content/data/Audios/5.wav"),
]


In [89]:
from torch.utils.data import Dataset
from PIL import Image
import torchaudio

class ImageAudioDataset(Dataset):
    def __init__(self, pairs, preprocess):
        self.pairs = pairs
        self.preprocess = preprocess

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        img_path, aud_path = self.pairs[idx]

        image = self.preprocess(Image.open(f'/content/Dataset/Images/{idx+1}.jpg').convert("RGB"))

        waveform, sr = torchaudio.load(f'/content/Dataset/Audios/{idx+1}.wav')

        return image, waveform


In [90]:
dataset = ImageAudioDataset(
    pairs=pairs,
    preprocess=preprocess   # from CLIP
)


In [91]:
from torch.utils.data import DataLoader

loader = DataLoader(
    dataset,
    batch_size=5,
    shuffle=True
)


In [96]:
def forward_batch(images,audios):
  img_emb = encod_img(images)
  aud_emb = encod_audio(audios)
  return img_emb,aud_emb


In [97]:
def contrastive_loss(img_emb, aud_emb):
    sim = img_emb @ aud_emb.T        # cosine similarity
    labels = torch.arange(sim.size(0)).to(sim.device)

    return torch.nn.functional.cross_entropy(sim, labels)


In [98]:
!pip install torchcodec


In [99]:
for epoch in range(num_epochs):
    for images, audios in loader:

        images = images.to(device)
        audios = audios.to(device)

        img_emb, aud_emb = forward_batch(images, audios)

        loss = contrastive_loss(img_emb, aud_emb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch}: loss = {loss.item():.4f}")


Epoch 0: loss = 2.3151
Epoch 1: loss = 2.3091
